In [ ]:
### Most the code is from https://github.com/Blandalytics/baseball_snippets/blob/main/run_expectancy.py
### Minor tweaks include filtering only inning <= 8, computing runs scored in the rest of the inning
### Install pybaseball package to import data
### Not necessary if you already have the data
# !pip install pybaseball -q
from pybaseball import statcast, cache

import datetime
import numpy as np
import pandas as pd
import matplotlib as mpl


### Load the data and preprocessing

In [ ]:
### Season Thresholds (roughly encapsulates the regular season for any given year)
this_year = 2025
season_start = datetime.datetime(this_year, 3, 1)
season_end = datetime.datetime(this_year, 11, 1)

In [ ]:
### Use pybaseball to load data for season
all_pitch_data = statcast(start_dt=season_start.strftime('%Y-%m-%d'),end_dt=season_end.strftime('%Y-%m-%d'))


In [ ]:
# Razzball player lookup
# You can download the csv from https://razzball.com/mlbamids/
razz = pd.read_csv("razzball.csv")

In [ ]:
razz_unique = razz[['Name', 'MLBAMID', 'Team']].drop_duplicates(subset='MLBAMID')

In [ ]:
all_pitch_data = all_pitch_data.merge(
    razz_unique,
    left_on="batter",
    right_on = 'MLBAMID',
    how="left"
)

In [ ]:
pd.set_option('display.max_columns', None)
all_pitch_data.head()

In [ ]:
# This shows that the code will capture non-regular season pitches
all_pitch_data['game_type'].value_counts()

In [ ]:
# Select only regular season games and innings before the 9th inning
pitch_data = all_pitch_data.loc[(all_pitch_data['game_type']=='R') & (all_pitch_data['inning']<=8)]

### Create new variables to facilitate computations, then compute

In [ ]:
### Create a base_state field, containing all of 1B/2B/3B
for base in ['1b','2b','3b']:
  pitch_data.loc[pitch_data['on_'+base].notna(),'on_'+base] = int(base[0])
  pitch_data['on_'+base] = pitch_data['on_'+base].astype('str').replace('<NA>','_')
pitch_data['base_state'] = pitch_data['on_1b']+' '+pitch_data['on_2b']+' '+pitch_data['on_3b']

In [ ]:
### Determine how many runs were scored for each inning
### I tweaked this from Bland's code
pitch_data['end_inning_score'] = pitch_data['bat_score'].groupby([pitch_data['game_pk'],pitch_data['inning'],pitch_data['inning_topbot']]).transform('max')
pitch_data['inning_runs'] = pitch_data['end_inning_score'].sub(pitch_data['bat_score']).astype('int')



In [ ]:
## This chunk helps pandas sort the base states
## It's optional and doesn't affect the calculations
from pandas.api.types import CategoricalDtype
base_state_cats = CategoricalDtype(
    ['_ _ _',
     '1 _ _',
     '_ 2 _',
     '_ _ 3',
     '1 2 _',
     '1 _ 3',
     '_ 2 3',
     '1 2 3'],
    ordered=True
)
pitch_data['base_state'] = pitch_data['base_state'].astype(base_state_cats)

In [ ]:
### Generate a dataframe for the 24 base-out states
### 3 outs x 8 base states
re_24_df = (pitch_data
            .dropna(subset=['events']) # Removes pitches that don't end a PA or otherwise change the base-out state
            .groupby(['game_year','base_state','outs_when_up']) # Find average runs at each base-out state
            ['inning_runs']
            .mean()
            .reset_index()
            .pivot(index=['base_state'], # Pivots data so that the 2 dimensions are outs and base state
                   columns='outs_when_up',
                   values='inning_runs')
            .copy()
)

In [ ]:
### Take a look at the table!
re_24_df.head(8)

### Compute Run Values for each plate appearance

In [ ]:
### Create a dictionary filled with the values from the table above for easy access
d_re = {0:0}
for outs in range(3):
  for base_state in re_24_df.index:
    d_re[base_state + str(outs)] = re_24_df.loc[base_state,outs]


In [ ]:
### Compute how many runs scored on the pitch
pitch_data['runs_on_play'] = pitch_data['post_home_score'] + pitch_data['post_away_score']- pitch_data['home_score'] - pitch_data['away_score']

In [ ]:
### Filter for only pitches where an event happened, gives us the last pitch of each PA
df = pitch_data[['game_pk', 'batter','base_state','outs_when_up','runs_on_play', 'events', 'inning', 'inning_topbot','Name','Team']].dropna(subset=['events'])

In [ ]:
### Find the state before and after the PA
df["state_before"] = df['base_state'].astype(str) + df['outs_when_up'].astype(str)

df["state_after"] = (
    df.groupby(["game_pk", "inning", "inning_topbot"])["state_before"].shift(1)
)


In [ ]:
### Many events are the end of an inning, which is currently an NA
df.isna().sum()

In [ ]:
### The dictionary above assigns d_re[0] = 0, the expected runs for the end of the inning, so fill nas with 0
df['state_after'] = df['state_after'].fillna(0)

In [ ]:
### Compute the Run Value of each play
df['re24'] = df['state_after'].map(d_re) - df['state_before'].map(d_re) + df['runs_on_play']

In [ ]:
### Find which batters led the majors in RE24 (sum)
df.groupby('batter')['re24'].sum().sort_values(ascending=False).head(10)

### Compute the Run Probability Matrix

In [ ]:
### Convert the number of runs scored the rest of the inning into a Boolean
### This tracks whether or not a run was scored the rest of the inning
pitch_data['any_runs'] = pitch_data['inning_runs'] > 0

In [ ]:
### Generate a dataframe for the 24 base-out states
### 3 outs x 8 base states
rp_24_df = (pitch_data
            .dropna(subset=['events']) # Removes pitches that don't end a PA or otherwise change the base-out state
            .groupby(['game_year','base_state','outs_when_up']) # Find proportion of times a run scored at each base-out state
            ['any_runs']
            .mean()
            .reset_index()
            .pivot(index=['base_state'], # Pivots data so that the 2 dimensions are outs and base state
                   columns='outs_when_up',
                   values='any_runs')
            .copy()
)

In [ ]:
### Take a look at the Run Probability Table
rp_24_df.head(8)